Dog Breed Prediction


  In this project, we will see how to use Keras and tensorflow to build, train, and test a Convolutional Neural Network capable of identifying the breed of a dog in a supplied image. This is a supervised learning problem, specifically for a multiclass classification problem.

In [ ]:
# to connect google colab with kaggle, download the kaggle.json file from kaggle and upload it here
from google.colab import files
files.upload()

In [ ]:
# install the kaggle API client
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/. kaggle, so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

These step were to setup the kaggle api

In [ ]:
# Creating a new working directory to store the data , and make this as working directory
!mkdir dog_dataset
%cd dog_dataset

In [ ]:
# Next step : to search the Kaggle for the required data set in order to store the datas
!kaggle datasets list -s dogbreedidfromcomp

The above step is to search the Kaggle for the required dataset using search option(-s) with title 'dogbreedidfromcomp'. We can also use different search options like searching competitions, notebooks, kernels, datasets, etc.

In [ ]:
# we will download this data from kaggle dataset
# Downloading dataset and coming out of directory
!kaggle datasets download catherinehorng/dogbreedidfromcomp
%cd ..

In [ ]:
# Un zip the downloaded data, remove the unusable files
!unzip dog_dataset/dogbreedidfromcomp.zip -d dog_dataset
!rm dog_dataset/dogbreedidfromcomp.zip
!rm dog_dataset/sample_submission.csv

In [ ]:
# the data is prepared, let's start building the model
# important librabry imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam

In [ ]:
# Next step : loading the labels data into the dataframe and view it
# read the labels.csv file and check the shape and records
labels_all = pd.read_csv('dog_dataset/labels.csv')
print(labels_all.shape)
labels_all.head()

Loading the labels data into dataframe and viewing it. Analysed that labels contains 10222 rows and 2 columns

In [ ]:
# Visualize the number of each breeds
breeds_all = labels_all['breed']
breed_counts = breeds_all.value_counts()
breed_counts.head()


Here we are finding out the count per class i.e. the total data in each class using value_counts() function

In [ ]:
# Selecting first 3 breeds ( limitation due to computation power)
CLASS_NAMES = ['scottish_deerhound', 'maltese_dog', 'bernese_mountain_dog']
labels = labels_all[labels_all['breed'].isin(CLASS_NAMES)]
labels = labels.reset_index()
labels.head()

As we are working with the classification dataset first we need to one hot encode the target value i.e. the classes. After that we will read images and convert them into numpy array and finally normalize the array

In [ ]:
# creating the numpy matrix with zeros
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(labels['breed'], classes = CLASS_NAMES)

# reading and converting image into numpy array and normalize the data set

for i in tqdm(range(len(labels))):
    img = image.load_img('dog_dataset/train/%s.jpg' % labels['id'][i], target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.

# Printing train image and one hot encode shape and size
print('Train Images shape: ',X_data.shape, 'size : {:,}'.format(X_data.size))
print('One hot encoded output shape:',Y_data.shape, 'size : {:,}'.format(Y_data.size))

In [ ]:
# Building the model
model = Sequential()  # considering the sequential model

# Conv2d, maxpool2d flatten are the layers for this model. Playing with the hyperparameters
# 1st layer = conv2d layer
model.add(Conv2D(filters = 64, kernel_size = (5, 5), activation='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))
# MaxPool layer finds out the most prominant features in the 2x2 matrix

model.add(Conv2D(filters = 16, kernel_size = (7, 7), activation='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5, 5), activation='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

# After setting up the layers, we will flatten the data as we need to apply the dense layer

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer = 'l2'))
model.add(Dense(64, activation='relu', kernel_regularizer = 'l2'))
model.add(Dense(len(CLASS_NAMES), activation='softmax'))

# softmax gives the probability of classes. So in a multiclass classification problem we generally
# takes softmax as the activation function.

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
model.summary()



So the above step was to create the network architecture for the model. Different types of layers according to their features namely Conv_2d (used to convolutional kernel that is convolved with the input layer to produce the output sensor) , max_pooling2d (it is a down sampling technique which takes out maximum value over the window defined by pool size), flatten( it flattens the input and creates the 1D output), Dense (Dense layer produce the output as  the dot product of the input and the kernel).

In [ ]:
# Splitting the data into training and testing data sets
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
# Splitting the training data into training and validation data sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

In [ ]:
# Taining the model / fit our model

epochs = 100
batch_size = 128

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val))

training the model on 100 epochs and batch size of 128, can try using more number of epochs to increase the accuracy. During each epochs we can see how the model is performing by viewing the training and validation accuracy

In [ ]:
# Plot the training history

plt.figure(figsize=(12, 12))
plt.plot(history.history['accuracy'], color ='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# predicting activity by the model
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

On changing the mnodel's hypermeters / hyperparameter tuning , we can achieve better accuracy

In [ ]:
# Plot the image to compare
plt.imshow(X_test[1, :, :, :])
plt.show()

# Finding the max value from prediction list and compareing original value vs predicted
print("Originally : ",labels['breed'][np.argmax(Y_test[1])])
print("Predicted : ",labels['breed'][np.argmax(Y_pred[1])])

In [ ]:
# saving the model
model.save('dog_breed_classification_model.h5') # h5 is the format for saving the model